In [1]:
from tkinter import *
import tkinter as tk
from tkinter import filedialog as fd
import math
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
import matplotlib.backends.tkagg as tkagg
from matplotlib.backends.backend_agg import FigureCanvasAgg
from PIL import ImageTk, Image
import skimage.data as data
import skimage.segmentation as seg
import skimage.filters as filters
import skimage.draw as draw
import skimage.color as color
from skimage.morphology import skeletonize as skel 
from skimage.morphology import disk
import imageio as iio
import scipy.ndimage as ndi

C:\Anaconda3\lib\site-packages\matplotlib\backends\tkagg.py:10: MatplotlibDeprecationWarning: 
The matplotlib.backends.tkagg module is deprecated.
  "3.0", "The matplotlib.backends.tkagg module is deprecated.")


In [2]:
def im_show(image, nrows=1, ncols=1, cmap='gray'):
    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(14, 14))
    ax.imshow(image, cmap='gray')
    ax.axis('off')
    return fig, ax

def pre_processing_dark(multi):    
    #thresholding, masking, filtering
    masked_multi = np.where(multi < np.percentile(multi, 99),0 , multi)
    mask_dilate = ndi.binary_dilation(masked_multi, iterations=1)
    mask_erosion = ndi.binary_erosion(mask_dilate, iterations=2)
    mask_closed = ndi.binary_closing(mask_erosion, iterations=3)
    mask_median = filters.median(mask_closed, disk(2.5))
    multi_final = mask_median
    
    return multi_final

def singlify(list_of_objects, n):
    for object in list_of_objects:
        #first part is to get labels and boxes 
        labels, nobs = ndi.label(object)
        boxes = ndi.find_objects(labels)

        #if there is only one object then no need to modify the picture
        if len(boxes)>1:
            #Then we have to compare their size and get the index of the biggest one
            obs = [np.array(object[boxes[i]]) for i in range(0,nobs-1)]
            obs_sizes = [ob.shape[0]*ob.shape[1] for ob in obs]
            n = int(np.where(obs_sizes == np.max(obs_sizes))[0])

            #now follows the mask that will filter everything but the biggest

            for i in range(0,n-1):
                if i != n:
                    object[boxes[i]].fill(0)
    return list_of_objects 

def backbone(objects):
    binaries = [np.where(ob > 0, 1, 0) for ob in objects]
    backbones = [skel(binary) for binary in binaries]
    binary_backbones = [np.where(ob ==True, 1, 0) for ob in backbones]
    return binary_backbones

def greyoverlap(singles, backbones):
    greyoverlap = [np.where(backbones[i] == 1, -80, 0) + singles[i] for i in range(0, len(singles))]
    return greyoverlap

In [3]:
im = iio.imread('multifil.PNG')
im = color.rgb2gray(im) 
#im_show(im)

In [4]:
im = pre_processing_dark(im)

In [5]:
labels, nobjects = ndi.label(im)

In [6]:
boxes = ndi.find_objects(labels)

In [7]:
objects = [np.array(im[boxes[i]]) for i in range(0, nobjects-1) if (np.array(im[boxes[i]]).shape[0]*np.array(im[boxes[i]]).shape[1] > 600)]

In [8]:
singles = singlify(objects, nobjects)

In [9]:
backbones = backbone(singles)